<a href="https://colab.research.google.com/github/amirmahdiKhosravi/end-to-end-MLOps-project/blob/main/End_toEnd_MLOps_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup for our project:

In [ ]:
%pip install "zenml[server]"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2

import IPython

IPython.Application.instance().kernel.do_shutdown(restart=True)

In [1]:
NGROK_TOKEN = "2Ze6COEBPZorYjW9mcX2NxGXkd0_5VjfzGigf1Q8wPQywZAmr"

In [2]:
from zenml.environment import Environment

if Environment.in_google_colab():
  !pip install pyngrok
  !ngrok authtoken {NGROK_TOKEN}

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


NumExpr defaulting to 2 threads.
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install zenml

ZenML setup:

In [ ]:
!rm -rf .zen
!zenml init

Example experimentation ML code:

In [5]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test():
  digits = load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(
      data, digits.target, test_size=0.2, shuffle=False
  )
  model = SVC(gamma=0.001)
  model.fit(X_train, y_train)
  test_acc = model.score(X_test, y_test)
  print(f"model accuracy: {test_acc}")

train_test()

model accuracy: 0.9583333333333334


Now we are to turn this simple experiment into ML pipeline with ZenML. this pipeline will automate the process of loading the data, training a model and evaluating it.

In [11]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer() -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "x_test"]
]:

  digits = load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(data, digits.target, test_size=0.2, shuffle = False)
  return X_train, X_test, y_train, y_test

@step
def SVC_trainer(X_train: np.ndarray, y_train : np.ndarray) -> ClassifierMixin:
  model = SVC(gamma=0.001)
  model.fit(X_train, y_train)
  return model

@step
def evaluator(X_test: np.ndarray, y_test : np.ndarray, model : ClassifierMixin) -> float:
  model_acc = model.score(X_test, y_test)
  print(f"Model accuracy: {model_acc}")
  return model_acc

Creating zenML pipeline:

In [12]:
from zenml import pipeline

@pipeline
def digits_pipeline():
  X_train, X_test, y_train, y_test = importer()
  model = SVC_trainer(X_train, y_train)
  evaluator(X_test, y_test, model)

Running zenML pipeline:

In [13]:
digits_SVC_pipeline = digits_pipeline()
#digits_SVC_pipeline.run(unlisted=True)

Initiating a new run for the pipeline: digits_pipeline.
Reusing registered version: (version: 1).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
Using cached version of importer.
Step importer has started.
Using cached version of SVC_trainer.
Linking artifact output to model None version None implicitly.
Step SVC_trainer has started.
Using cached version of evaluator.
Linking artifact output to model None version None implicitly.
Step evaluator has started.
Run digits_pipeline-2023_12_17-22_05_39_817528 has finished in 0.441s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


now with following code we can visualize zenML the pipeline in zenML dashboard. the username is loca and the password should be left empty.

In [ ]:
from zenml.environment import Environment

def start_zenml_dashboard(port=8237):
  if Environment.in_google_colab():
    from pyngrok import ngrok

    public_url = ngrok.connect(port)
    print(f"use this url instead: {public_url}")
    !zenml up --blocking --port {port}

  else:
    !zenml up --port {port}

start_zenml_dashboard()

INFO:pyngrok.ngrok:Opening tunnel named: http-8237-7f6f51c1-90a7-4e97-bd20-31cd3b2329b1


Opening tunnel named: http-8237-7f6f51c1-90a7-4e97-bd20-31cd3b2329b1


INFO:pyngrok.process.ngrok:t=2023-12-17T22:23:59+0000 lvl=info msg="no configuration paths supplied"


t=2023-12-17T22:23:59+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2023-12-17T22:23:59+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

t=2023-12-17T22:23:59+0000 lvl=info msg=start pg=/api/tunnels id=5453a2b8cdf66fd7


INFO:pyngrok.process.ngrok:t=2023-12-17T22:23:59+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-7f6f51c1-90a7-4e97-bd20-31cd3b2329b1 addr=http://localhost:8237 url=https://aa90-34-147-59-123.ngrok-free.app


use this url instead: NgrokTunnel: "https://aa90-34-147-59-123.ngrok-free.app" -> "http://localhost:8237"
t=2023-12-17T22:23:59+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-7f6f51c1-90a7-4e97-bd20-31cd3b2329b1 addr=http://localhost:8237 url=https://aa90-34-147-59-123.ngrok-free.app


INFO:pyngrok.process.ngrok:t=2023-12-17T22:23:59+0000 lvl=info msg=end pg=/api/tunnels id=5453a2b8cdf66fd7 status=201 dur=106.648958ms


t=2023-12-17T22:23:59+0000 lvl=info msg=end pg=/api/tunnels id=5453a2b8cdf66fd7 status=201 dur=106.648958ms
NumExpr defaulting to 2 threads.
Updating the local ZenML server.
⠋ Stopping service 'LocalZenServer[9a4ac2eb-dad5-4747-8f30-5e622a95d211] (type: zen_server, flavor: 
local)'.

Starting ZenML Server as blocking process... press CTRL+C once to stop it.
INFO:     Started server process [32915]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8237 (Press CTRL+C to quit)
